In [ ]:
import sys

import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.tri as tri
import matplotlib.colors as colors

import cfgrib
import xesmf

from scipy.io import FortranFile
from scipy import interpolate as intr
from scipy import special as sps

import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import pyproj

import importlib
import glob
import copy
import time
#from cfgrib.xarray_store import open_dataset


# import modules in other directories
sys.path.append('../Utils/')
import MyConstants as Con

pi=Con.pi()


In [ ]:
#ChatGPT says you can calculate spherical harmonics 'm' and 'n' using nsp and nc2 
# according to:
#nsp, nc2 = arr.shape
#n = np.arange(nsp)[:, np.newaxis]
#m = np.arange(nc2)
#m = np.tile(m, (nsp, 1)) - n

#Well the above doesn't seem to work so well after more prodding ChatGPT says:

#ChatGPT couldn't seem to understand what nc2 is.

#nsp, nc2 = arr.shape
#m = np.zeros( (nsp,nc2) )
#n = np.zeros( (nsp,nc2) )

#for i in range(nsp):
#    for j in range(-i, i+1):
#        k = i*(i+1)//2 + j
#        m[i, k] = j
#        n[i, k] = i

In [ ]:
#foo='/glade/work/juliob/ERA5-proc/Data/initdata_01.grib'
foo='/glade/work/juliob/ERA5-proc/Data/initdata_01.nc'
#doo=xr.open_dataset(foo , engine='cfgrib' , backend_kwargs={'filter_by_keys': {'typeOfLevel': 'model'}})
#doo=xr.open_dataset(foo , engine='cfgrib' )# , backend_kwargs={'errors': 'ignore'})
ds=xr.open_dataset(foo) #,engine='cfgrib',backend_kwargs={'indexpath': ''})


In [ ]:
#print(len(doo['time']))
print(list(ds.variables))
print(list(ds.attrs))
print(list(ds.dims))
ds.dims['nsp']
print(np.shape(ds['u']))
print(ds['u'].dims)
nt,nz,nsp,nc2 = np.shape(ds['u'] )
print( 640*640/2)
print( nsp - 640*640/2 )
MM=639
nnn=MM**2 + 2*MM + 1
print(nnn,nsp*2,nsp*2-nnn)

In [ ]:
c=0
mm=np.zeros( nsp )
nn=np.zeros( nsp )
ics = np.zeros( nsp , dtype=int )
for q in np.arange(0,640):
    for p in np.arange(q,640):
        nn[c]=p
        mm[c]=q
        ics[c] = c
        c=c+1
print(c)

nmax = 90
oo=np.where( nn <= nmax)
nnT=nn[oo[0]]
mmT=mm[oo[0]]
icsT=ics[oo[0]]

In [ ]:
nspT=len(nnT)
print(nspT)

In [ ]:
plt.scatter(mm,nn)
plt.scatter(mmT,nnT)
plt.xlim(0,150)
plt.ylim(0,150)

In [ ]:
u=ds['u'].values
yurg=np.zeros( (ny,nx) )
Fc=u[0,100,:,:]
for i in np.arange( nspT ):
    ii=icsT[i]
    ymn = sps.sph_harm( mmT[i],nnT[i], Az, Po)
    yurg = yurg + (Fc[ii,0]+Fc[ii,1]*1.j)*ymn

In [ ]:
#plt.contour( az, po,  yurg  )
print(np.max(yurg))
#plt.title( "Nmax="+str(nmax)+" modes ="+str(nspT) )


In [ ]:
plt.contour( az, po,  yurg  )
plt.title( "Nmax="+str(nmax)+" modes ="+str(nspT) )

In [ ]:
plt.plot( ds['u'].values[0,100,:,0])

In [ ]:
data_arr = xr.DataArray(ds['u'], coords=[ds['time'], 
                        ds['lev'], ds['nsp'], ds['nc2']], attrs=ds.attrs)

In [ ]:
# Define the projection for the spectral grid
input_grid = \
xesmf.backend.Spectral(data=data_arr, dims=['lat', 'lev'], nsp=ds.attrs['nsp'], ltrip=0 ) #era5_data.attrs['ltrip'])

In [ ]:
# Convert the spectral data to grid point data
grid_data = pyproj.spectral_to_gridpoint(ds['u'], spectral_proj)

In [ ]:
era5_data = xr.open_dataset(foo)

era5_wcs = era5_data['u'].attrs['wcs']

In [ ]:
ds